In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import pylab as pl
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv("data.csv")
df = pd.DataFrame(data) # Converting data to Panda DataFrame
df.head() #gives statistics about the columns of the dataframe

,dataSet,Input,instructionCount,ilp32,ilp256,ilp2048,ilp65536,memops,ctrlops,intops,...,lbdiv64,lbdiv128,lbdiv256,lbdiv512,lbdiv1024,probsize,Class,CPU Time,GPU Time,Ratio
0,MVT_8192: 51-63,8192,26233453,6.498181,13.011978,13.078701,15.976306,0.820424,0.051388,0.846086,...,0.000000,0.000000,0.000000,0.000000,0.000000,74,Y,11.144549,3.324654,3.352093
1,MVT_16384: 51-63,16384,104658385,6.499089,13.005995,13.039291,14.338086,0.820468,0.051335,0.846120,...,0.000000,0.000000,0.000000,0.000000,0.000000,148,Y,5.509123,1.766470,3.118719
2,MVT_20000: 51-63,20000,156024013,6.157181,13.004910,13.032163,14.076138,0.820476,0.051325,0.846126,...,0.000000,0.000000,0.000000,0.000000,0.000000,181,Y,1.165420,0.658309,1.770324
3,syrk_m_2048: 59-70,2048,23223871910,6.749968,9.031183,9.332883,14.100434,0.666661,0.074120,0.666661,...,0.000570,0.001140,0.002280,0.004560,0.009120,18,Y,15.701999,14.617724,1.074175
4,syrk_m_1024: 59-70,1024,2996927698,6.999141,9.398286,10.028998,15.750233,0.642840,0.071516,0.678563,...,0.002209,0.004417,0.008835,0.017669,0.035338,9,Y,1.954343,1.856112,1.052923


## Weight all the features by instruction count rather than a separate instruction

In [3]:
X_ = df

Weighted_X = X_[["ilp32",'ilp256','ilp2048','ilp65536','memops','ctrlops','intops','flops',
 'coldref fraction',
 'resuseDist2 fraction',
 'sfp',
 'dfp',
 'noconflict',
 'broadCast',
 'coalesced',
 'shMemBw',
 'gMemBw',
 'blocks',
 'pages',
 'lipRate',
 'mulf',
 'divf',
 'specialFn',
 'lbdiv16',
 'lbdiv32',
 'lbdiv64',
 'lbdiv128',
 'lbdiv256',
 'lbdiv512',
 'lbdiv1024']].multiply(df["instructionCount"], axis="index")


Weighted_X.shape

X_train = Weighted_X
Y_train = df['Class']

## Splitting Data to Training set and Test set 0.1%

In [4]:
%%script false
Y_train = df['Class']

#seperate traindata and test data here
X_train, X_test, Y_train, Y_test = train_test_split(Weighted_X,Y_train ,test_size=0.1)


In [5]:
dataTest = pd.read_csv("test.csv")
dfTest = pd.DataFrame(dataTest) # Converting data to Panda DataFrame

X_Test = dfTest

Weighted_XTest = X_Test[["ilp32",'ilp256','ilp2048','ilp65536','memops','ctrlops','intops','flops',
 'coldref fraction',
 'resuseDist2 fraction',
 'sfp',
 'dfp',
 'noconflict',
 'broadCast',
 'coalesced',
 'shMemBw',
 'gMemBw',
 'blocks',
 'pages',
 'lipRate',
 'mulf',
 'divf',
 'specialFn',
 'lbdiv16',
 'lbdiv32',
 'lbdiv64',
 'lbdiv128',
 'lbdiv256',
 'lbdiv512',
 'lbdiv1024']].multiply(dfTest["instructionCount"], axis="index")


Weighted_XTest.shape

X_test = Weighted_XTest
Y_test = dfTest['Class']

X_test.head(7)


,ilp32,ilp256,ilp2048,ilp65536,memops,ctrlops,intops,flops,coldref fraction,resuseDist2 fraction,...,mulf,divf,specialFn,lbdiv16,lbdiv32,lbdiv64,lbdiv128,lbdiv256,lbdiv512,lbdiv1024
0,2.021750e+10,3.060798e+10,7.587934e+10,1.179372e+12,5.984552e+09,1.135358e+09,6.498338e+09,4.900000e+06,4.158411e+05,4.850308e+09,...,0.000000e+00,0.0,0.0,5.625914e+07,6.755249e+07,8.000426e+07,8.001699e+07,8.004246e+07,8.009314e+07,8.019773e+07
1,8.564380e+10,5.514055e+10,1.047456e+11,2.654958e+12,7.931205e+09,3.965542e+09,8.922600e+09,0.000000e+00,5.036227e+07,8.055026e+09,...,0.000000e+00,0.0,0.0,1.533689e+06,3.041602e+06,6.066636e+07,1.213290e+08,2.435859e+08,4.871718e+08,9.704772e+08
2,4.196546e+10,2.701887e+10,5.132534e+10,1.300929e+12,3.886290e+09,1.943116e+09,4.372074e+09,0.000000e+00,2.467751e+07,3.946963e+09,...,0.000000e+00,0.0,0.0,7.497033e+05,1.488790e+06,2.972652e+07,5.945120e+07,1.191770e+08,2.385720e+08,4.758164e+08
3,3.358006e+10,2.162005e+10,4.106968e+10,1.040982e+12,3.109745e+09,1.554849e+09,3.498460e+09,0.000000e+00,1.974653e+07,3.158294e+09,...,0.000000e+00,0.0,0.0,6.004213e+05,1.191304e+06,2.378666e+07,4.757186e+07,9.536341e+07,1.909013e+08,3.807404e+08
4,1.313592e+08,8.457375e+07,1.606572e+08,4.072134e+09,1.216476e+07,6.082289e+06,1.368535e+07,0.000000e+00,7.724492e+04,1.235468e+07,...,0.000000e+00,0.0,0.0,2.347067e+03,4.658803e+03,9.304916e+04,1.860926e+05,3.730446e+05,7.467716e+05,1.489388e+06
5,1.386610e+11,2.086818e+11,2.156520e+11,3.258744e+11,1.797425e+10,1.712948e+09,1.797425e+10,4.279443e+09,2.022176e+05,1.430673e+10,...,9.243783e+09,0.0,0.0,3.309551e+06,6.619105e+06,1.323821e+07,2.647642e+07,5.295283e+07,1.059057e+08,2.118118e+08
6,1.733858e+10,2.618103e+10,2.799611e+10,4.415642e+10,2.247513e+09,2.143277e+08,2.247513e+09,5.350871e+08,5.057270e+04,1.789011e+09,...,1.155835e+09,0.0,0.0,1.654752e+06,3.309504e+06,6.619008e+06,1.323802e+07,2.647603e+07,5.295206e+07,1.059062e+08


## Reading Test Data from unseen Data (Guaranteed_)

In [6]:
%%script false
data_test = pd.read_csv("test.csv")
df_test = pd.DataFrame(data_test) # Converting data to Panda DataFrame
df_test.head() #gives statistics about the columns of the dataframe

In [7]:
%%script false
X_test = df_test
Weighted_XTest = X_test[["ilp32",'ilp256','ilp2048','ilp65536','memops','ctrlops','intops','flops',
 'coldref fraction',
 'resuseDist2 fraction',
 'sfp',
 'dfp',
 'noconflict',
 'broadCast',
 'coalesced',
 'shMemBw',
 'gMemBw',
 'blocks',
 'pages',
 'lipRate',
 'mulf',
 'divf',
 'specialFn',
 'lbdiv16',
 'lbdiv32',
 'lbdiv64',
 'lbdiv128',
 'lbdiv256',
 'lbdiv512',
 'lbdiv1024']].multiply(df_test["instructionCount"], axis="index")


Weighted_XTest.head()

## Class Distribution

In [8]:
yes = len(data[data.Class == "Y"])
No = len(data[data.Class == "N"])

print yes
print No

69
23


In [9]:
#Y_train = df['Class']

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)


#Y_test = df_test['Class']
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

print len(X_train)
print len(X_test)

92
15


## Ensembled Bagging Decison Tree Classifier

In [10]:
baggingClassifier = ensemble.BaggingClassifier(n_estimators=11,bootstrap_features=True,n_jobs=-1)
baggingClassifier.fit(X_train,Y_train)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=True, max_features=1.0, max_samples=1.0,
         n_estimators=11, n_jobs=-1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

## Cross validating the model

In [11]:
scores = cross_val_score(baggingClassifier, X_train, Y_train)
print scores

[0.74193548 0.61290323 0.53333333]


In [12]:
prediction = baggingClassifier.predict(X_test)
print prediction
print Y_test

['N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N']
['N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N']


In [13]:
accuracy_score(Y_test, prediction)

0.6666666666666666

## Random Forest Classifier


In [14]:
#similiar to ensembling techniques
#use majority vote from the forest
#can handle missing values in the dataset
#samples are used with replacement

randomForest = RandomForestClassifier(n_estimators=11,n_jobs=-1,random_state=0)
scores = cross_val_score(randomForest, X_train, Y_train)
print scores


[0.61290323 0.67741935 0.66666667]


In [15]:
randomForest.fit(X_train,Y_train)
prediction = randomForest.predict(X_test)

print prediction
print Y_test

['N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y']
['N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N']


In [16]:
accuracy_score(Y_test, prediction)

0.5333333333333333

In [17]:
importances = pd.DataFrame({'feature':Weighted_X.columns,'importance':np.round(randomForest.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')

importances.head(32)

,importance
feature,
coldref fraction,0.123
blocks,0.102
lbdiv256,0.074
noconflict,0.055
ilp2048,0.053
pages,0.049
flops,0.049
intops,0.040
ilp256,0.039


## Executing Recursive Feature Selection to choose important features <- Check

## Nearest Neighbor Method

In [18]:
neighbor = KNeighborsClassifier(n_neighbors=11,weights='distance',algorithm='auto')
scores = cross_val_score(neighbor, X_train, Y_train)
print scores

[0.64516129 0.74193548 0.66666667]


In [19]:
neighbor.fit(X_train,Y_train)
prediction = neighbor.predict(X_test)

print prediction
print Y_test

['N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N']
['N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N']


In [20]:
accuracy_score(Y_test, prediction)

0.6

## SVM Classifier 

In [21]:
svmClassifier = SVC(kernel='rbf')
scores = cross_val_score(svmClassifier, X_train, Y_train)
print scores


[0.74193548 0.74193548 0.76666667]


In [22]:
svmClassifier.fit(X_train,Y_train)
prediction = svmClassifier.predict(X_test)

print prediction
print Y_test

['N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y']
['N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N']


In [23]:
accuracy_score(Y_test, prediction)

0.26666666666666666

## Ratio Method Classifier

In [25]:
Y_train = df['Ratio']

Y_train = np.asarray(Y_train)

## Bagging Regressor 

In [31]:
baggingRegressor = BaggingRegressor(n_estimators=10)
baggingRegressor.fit(X_train,Y_train)

predictions = baggingRegressor.predict(X_test)

results = []

for y in predictions:
    if y >1:
        results.append('Y')
    else:
        results.append('N')
print results
print Y_test

['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']
['N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N']


## RandomForest Regressor

In [34]:
randomforestRegressor = RandomForestRegressor(n_estimators=5,n_jobs=-1)
randomforestRegressor.fit(X_train,Y_train)

predictions = randomforestRegressor.predict(X_test)
results = []

for y in predictions:
    if y >1:
        results.append('Y')
    else:
        results.append('N')
print results
print Y_test
print predictions

['N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']
['N' 'N' 'N' 'N' 'N' 'Y' 'Y' 'Y' 'N' 'N' 'N' 'N' 'N' 'N' 'N']
[ 0.99178778 25.63944833 25.63944833 26.15216396  1.76957969  8.31240716
  6.78229883  2.27687946  1.18634603  2.16810826  2.08214879  2.08214879
  2.01470761  1.54059591  2.34985814]
